# Capstone Project Notebook

# Part I

## 1. Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into pandas dataframe.

### Using Xpath and Pandas to transform the data in the table on the Wikipedia page into pandas dataframe

### Import Libraries

<p>We will need requests for getting the HTML contents of the website and lxml.html for parsing the relevant fields. Finally, we will store the data on a Pandas Dataframe.</p>

In [1]:
import pandas as pd
import requests
import lxml.html as lh

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')


<p>For sanity check, ensure that all the rows have the same width. If not, we probably got something more than just the table.</p>

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Looks like all our rows have exactly 3 columns. This means all the data collected on tr_elements are from the table.

### Parse Table Header

Next, let’s parse the first row as our header.

In [4]:
#col.append((name,[]))
    
    #print("Total score for %s is %s" % (name, score))
#tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ("%d : %s" % (i,name))
    col.append((name,[]))      
    
           

1 : Postcode
2 : Borough
3 : Neighbourhood



### Creating Pandas DataFrame

Each header is appended to a tuple along with an empty list.

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    #T=tr_elements[j]
    f=0
    for field in tr_elements[j]:
        f+=1
        name=field.text_content()
        if name=='Not assigned':
            break
        print(name)

M1A
M2A
M3A
North York
Parkwoods

M4A
North York
Victoria Village

M5A
Downtown Toronto
Harbourfront

M6A
North York
Lawrence Heights

M6A
North York
Lawrence Manor

M7A
Downtown Toronto
Queen's Park

M8A
M9A
Etobicoke
Islington Avenue

M1B
Scarborough
Rouge

M1B
Scarborough
Malvern

M2B
M3B
North York
Don Mills North

M4B
East York
Woodbine Gardens

M4B
East York
Parkview Hill

M5B
Downtown Toronto
Ryerson

M5B
Downtown Toronto
Garden District

M6B
North York
Glencairn

M7B
M8B
M9B
Etobicoke
Cloverdale

M9B
Etobicoke
Islington

M9B
Etobicoke
Martin Grove

M9B
Etobicoke
Princess Gardens

M9B
Etobicoke
West Deane Park

M1C
Scarborough
Highland Creek

M1C
Scarborough
Rouge Hill

M1C
Scarborough
Port Union

M2C
M3C
North York
Flemingdon Park

M3C
North York
Don Mills South

M4C
East York
Woodbine Heights

M5C
Downtown Toronto
St. James Town

M6C
York
Humewood-Cedarvale

M7C
M8C
M9C
Etobicoke
Bloordale Gardens

M9C
Etobicoke
Eringate

M9C
Etobicoke
Markland Wood

M9C
Etobicoke
Old Burnhamt

In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        
        data=t.text_content() 
        #Check if row is empty
        if i>0:
            #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Just to be sure, let’s check the length of each column. Ideally, they should all be the same.

In [7]:
[len(C) for (title,C) in col]

[287, 287, 287]

Perfect! This shows that each of our 3 columns has exactly 287 values.

Now we are ready to create the DataFrame:

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

Looking at the top 5 cells on the DataFrame:

In [9]:
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


## 2. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

### Renaming column names 

In [10]:
df.rename(columns={'Postcode': 'PostalCode', 'Borough': 'Borough', 'Neighbourhood\n': 'Neighborhood'}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Strip Trailing Space of the Neighbourhood column

In [11]:
df['Neighborhood'] = df['Neighborhood'].str.rstrip()
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [13]:
df.shape

(287, 3)

## 3.Only processing the cells that have an assigned borough. Ignoring cells with a borough that is Not assigned.

In [14]:
#df = df[df['Borough'] != 'Not assigned'] 
# Get names of indexes for which column Borough has value "Not assigned"
indexNames = df[df['Borough'] =='Not assigned'].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)        
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [15]:
df.shape

(210, 3)

## 4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [16]:
#for index_label, row_series in df.iterrows():
    #if df.at[index_label , 'Neighborhood'] == 'Not assigned':
        #df.at[index_label , 'Neighborhood'] = df.at[index_label , 'Borough']
        
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
df.head(10)
 

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [17]:
df.shape

(210, 3)

## 5.Rows with same postalcode will combined into one row with the neighborhoods separated with a comma

In [18]:

#df['Neighborhood'] = df.groupby(['PostalCode'])['Neighborhood'].transform(lambda x: ','.join(x))
#df.drop_duplicates(inplace=True)

result = df.groupby(['PostalCode', 'Borough'], sort=False).agg(', '.join)
df_new = result.reset_index()
df_new.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## 6.Use the .shape method to print the number of rows of your dataframe.

In [19]:
df_new.shape

(103, 3)

# Part 2

### Using csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [20]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data

In [21]:
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
df_lon_lat.columns = ['PostalCode', 'Latitude', 'Longitude']
df_lon_lat

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [23]:
df_lon_lat.shape

(103, 3)

In [24]:
dfinal = df_new.merge(df_lon_lat, on="PostalCode", how = 'inner')
dfinal

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
